# <u>Bengals Data ETL</u>

### Establish Package Imports

In [4]:
import boto3
import pandas as pd

### Create Connection Class for AWS S3 Bucket

Notes:
- AWS Credentials were setup with Environment Variables

In [44]:
class Data_Connection:
    def __init__(self):
        self.aws_connection = boto3.Session()
        self.s3_connection = self.aws_connection.client('s3')
    
    def check_bucket_connection(self):
        print("Configurating Connection to S3 Bucket...")
        
        try:
            response = self.s3_connection.list_objects(Bucket='mindex-data-analytics-code-challenge')
            print("Connection Established!")
            
        except Exception as e:
            print(e)
            
        
        

In [45]:
connection = Data_Connection()
connection.check_bucket_connection()

Configurating Connection to S3 Bucket...
Connection Established!
